# Trabalho Prático 2 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Criptosistemas pós-quânticos PKE/KEM* - NTRU

Implementar o esquema NTRU em classes *Python/SageMath* apresentando, para cada um, as versões **KEM-IND-CPA** e **PKE-IND-CCA**.

Nota: Baseado no documento NTRU (https://ntru.org/f/ntru-20190330.pdf)

Conjuntos de parâmetros ntru-hrss específicos são denotados ntruhrss(n), por exemplo **ntruhrss701** é ntru-hrss com **n = 701**. O conjunto de parâmetros ntru-hrss recomendado é ntruhrss701, de acordo com o que foi submetido na 3ªronda do PQC.

### NTRU - PKE


In [1]:
import random, hashlib
import numpy as np

# Baseado no esquema da página 25 do documento https://ntru.org/f/ntru-20190330.pdf
# https://latticehacks.cr.yp.to/ntru.html

class NTRU_PKE(object):
    
    def __init__(self, N=821, Q=4096, D=495, timeout=None):
        
        # Todas as inicializações de parâmetros são baseadas na submissao com os parâmetros do ntruhps4096821, onde n = 821
        self.n = N
        self.q = Q
        self.d = D
        
        # Definição dos aneis
        Zx.<x>  = ZZ[]
        self.Zx = Zx
        Qq = PolynomialRing(GF(self.q), 'x')
        x = Zx.gen()
        y = Qq.gen()
        R = Zx.quotient(x^self.n-1)
        self.R = R
        Rq = QuotientRing(Qq, y^self.n-1)
        self.Rq = Rq
        
          
    # Gera uma string de bits com tamanho size e d 1's
    def randomBitString(self, size):
        
        # Gera uma sequencia de n bits aleatorios
        u = [random.choice([0,1]) for i in range(size)]
        # Mistura os valores da lista, so para aumentar a aleatoriedade
        random.shuffle(u)
        return u
    
    
    # Verifica se um polinomio e ternario
    def isTernary(self, f):
        
        res = True
        v = list(f)
        for i in v:
            if i > 1 or i < -1:
                res = False
                break
        return res
    
    
    # Produz o polinomio f modulo q. Mas, em vez de ser entre 0 e q-1, fica entre -q/2 e q/2-1
    def balancedmod(self, f, q):
        
        g = list(((f[i] + q//2) % q) - q//2 for i in range(self.n))
        return self.Zx(g)
    
    
    # Produz a inversa de um polinomio f modulo x^n-1 modulo p, em que p é um numero primo. 
    def invertmodprime(self, f, p):
        
        T = self.Zx.change_ring(Integers(p)).quotient(x^self.n-1)
        return self.Zx(lift(1 / T(f)))
    
    
    # Como a funcao de cima, mas o q aqui é uma potencia de 2
    def invertmodpowerof2(self, f, q):
        
        assert q.is_power_of(2)
        g = self.invertmodprime(f, 2)
        while True:
            r = self.balancedmod(self.R(g*f), q)
            if r == 1:
                return g
            g = self.balancedmod(self.R(g*(2 - r)), q)
    
    
    # Gera um polinomio ternario
    def Ternary(self, bit_string):
        
        # cria um array
        result = []
        # Itera d vezes
        for j in range(self.d):
            # Se o bit for 0, acrescenta 1, senao -1
            if bit_string[j] == 0:
                result += [1]
            elif bit_string[j] == 1:
                result += [-1]
        # Preenche com 0's o array restante
        result += [0]*(self.n-self.d)
        # Mistura os valores do array
        random.shuffle(result)

        return self.Zx(result)
    
    
    # Gera um polinomio f em Lf (no nosso caso, em T+) e um polinomio g em Lg (no nosso caso, em {φ1.v : v € T+})
    def Sample_fg(self, seed):
        
        x = self.R.gen()
        
        # Parse de fg_bits em f_bits||g_bits
        f_bits = seed[:self.d]
        g_bits = seed[self.d:]
        
        # Definir f = Ternary_Plus(f_bits)
        f = self.Ternary(f_bits)
        # Definir g0 = Ternary_Plus(g_bits)
        g = self.Ternary(g_bits)
        
        return (f,g)
    
    
    # Gera um polinomio r em Lr (no nosso caso, em T) e um polinomio m em Lm (no nosso caso, em T)
    def Sample_rm(self, coins):
        
        # sample_iid_bits = 8*n - 8
        sample_iid_bits = 8*self.n - 8
        
        # Parse de rm_bits em r_bits||m_bits
        r_bits = coins[:sample_iid_bits]
        m_bits = coins[sample_iid_bits:]
        
        # Set r = Ternary(r_bits)
        r = self.Ternary(r_bits)
        # Set m = Ternary(m_bits)
        m = self.Ternary(m_bits)
        
        return (r,m)
    
    
    # Funcao usada para gerar o par de chaves pública e privada
    def geraChaves(self, seed):
        
        while True:
            try:
                (f,g) = self.Sample_fg(seed)
                # fp <- (1/f) mod(3;φn)
                fp = self.invertmodprime(f, 3)
                # fq <- (1/f) mod (q;φn)
                fq = self.invertmodpowerof2(f, self.q)
                # gq <- (1/f) mod (q;φn) (so para garantir que h e invertivel)
                gq = self.invertmodpowerof2(g, self.q)
                # h <- (3.g.fq) mod(q;φ1φn)
                h = self.balancedmod(3*self.R(g*fq), self.q)
                # hq <- (1/f) mod (q;φn)
                hq = self.invertmodpowerof2(h, self.q)
                break
            except:
                # Para que a nova iteracao tenha uma nova seed
                seed = self.randomBitString(2*self.d)
                pass
        
        return {'sk' : (f,fp,hq) , 'pk' : h}

    
    # Recebe como parametros a chave publica e o tuplo (r,m)
    def cifra(self, h, rm):
    
        r = rm[0]
        m = rm[1]
        # c <- (r.h + m') mod (q,φ1φn)
        c = self.balancedmod(self.R(h*r) + m, self.q)
        
        return c
    
    
    # Recebe como parametros a chave privada (f,fp,hq) e ainda o criptograma
    def decifra(self, sk, c):
        
        # a <- (c.f) mod(q,φ1φn)
        a = self.balancedmod(self.R(c*sk[0]), self.q)
        # m <- (a.fp) mod(3,φn)
        m = self.balancedmod(self.R(a * sk[1]), 3)
        # r <- ((c-m').hq) mod(q,φn)
        aux = (c-m) * sk[2]
        r = self.balancedmod(self.R(aux), self.q)
        # Se os polinomios nao forem ternarios, retorna erro
        if not self.isTernary(r) and not self.isTernary(m):
            (0,0,1)
        return (r,m,0)
        

#### Testagem da classe definida acima:

In [13]:
# Parametros do NTRU (ntruhps4096821)
N=821
Q=4096
D=495

# Inicializacao da classe
ntru = NTRU_PKE(N,Q,D)

print("[Teste da cifragem e decifragem]")
keys = ntru.geraChaves(ntru.randomBitString(2*D))
rm = ntru.Sample_rm(ntru.randomBitString(11200))

c = ntru.cifra(keys['pk'], rm)

rmDec = ntru.decifra(keys['sk'], c)

if rmDec[0] == rm[0] and rmDec[1] == rm[1] and rmDec[2] == 0:
    print("As mensagens e os r's são iguais!!!!")
else:
    print("A decifragem falhou!!!!")

[Teste da cifragem e decifragem]
As mensagens e os r's são iguais!!!!


### NTRU - KEM

In [14]:
import random, hashlib
import numpy as np

# Baseado no esquema da página 25 do documento https://ntru.org/f/ntru-20190330.pdf
# https://latticehacks.cr.yp.to/ntru.html

class NTRU_KEM(object):
    
    def __init__(self, N=821, Q=4096, D=495, timeout=None):
        
        # Todas as inicializações de parâmetros são baseadas na submissao com os parâmetros do ntruhps4096821, onde n = 821
        self.n = N
        self.q = Q
        self.d = D
        
        # inicializacao da instancia NTRU_PKE
        self.pke = NTRU_PKE(self.n, self.q, self.d)
    
    
    #função para calcular o hash (recebe dois polinomios)
    def Hash1(self, e0, e1):
        
        ee0 = reduce(lambda x,y: x + y.binary(), e0.list() , "")
        ee1 = reduce(lambda x,y: x + y.binary(), e1.list() , "")
        m = hashlib.sha3_256()
        m.update(ee0.encode())
        m.update(ee1.encode())
        return m.hexdigest()
    
    
    #função para calcular o hash (recebe uma string de bits e um polinomio)
    def Hash2(self, e0, e1):
        
        ee1 = reduce(lambda x,y: x + y.binary(), e1.list() , "")
        m = hashlib.sha3_256()
        m.update(e0.encode())
        m.update(ee1.encode())
        return m.hexdigest()
    
    
    # Funcao usada para gerar o par de chaves pública e privada(acrescenta ao geraChaves1() um s)
    def geraChaves(self, seed):
        
        # ((f,fp),h) <- KeyGen'()
        keys = self.pke.geraChaves(seed)
        # s <-$ {0,1}^256
        s = ''.join([str(i) for i in self.pke.randomBitString(256)])
        
        # return ((f,fp,hq,s),h)
        return {'sk' : (keys['sk'][0],keys['sk'][1],keys['sk'][2],s) , 'pk' : keys['pk']}
        
    
    # Funcao que serve para encapsular a chave que for acordada a partir de uma chave publica
    def encapsula(self, h):
        
        # coins <-$ {0,1}^256
        coins = self.pke.randomBitString(256)
        # (r,m) <- Sample_rm(coins)
        (r,m) = self.pke.Sample_rm(self.pke.randomBitString(11200))
        # c <- Encrypt(h,(r,m))
        c = self.pke.cifra(h, (r,m))
        # k <- H1(r,m)
        k = self.Hash1(r,m)
        
        return (c,k)
     
    # Funcao usada para desencapsular uma chave, a partir do seu "encapsulamento" e da chave privada
    def desencapsula(self, sk, c):
        
        # (r,m,fail) <- Decrypt((f,fp,hq),c)
        (r,m,fail) = self.pke.decifra((sk[0], sk[1], sk[2]), c)
        # k1 <- H1(r,m)
        k1 = self.Hash1(r,m)
        # k2 <- H2(s, c)
        k2 = self.Hash2(sk[3],c)
        # if fail = 0 return k1 else return k2
        if fail == 0:
            return k1
        else:
            return k2
        
        

#### Testagem da classe definida acima:

In [15]:
# Parametros do NTRU (ntruhps4096821)
N=821
Q=4096
D=495

# Inicializacao da classe
ntru = NTRU_KEM(N,Q,D)

print("[Teste do encapsulamento e desencapsulamento]")
keys1 = ntru.geraChaves(ntru.pke.randomBitString(2*D))

(c,k) = ntru.encapsula(keys1['pk'])
print("Chave = " + k)

k1 = ntru.desencapsula(keys1['sk'], c)
print("Chave = " + k1)

if k == k1:
    print("A chave desencapsulada é igual à resultante do encapsulamento!!!")
else:
    print("O desencapsulamento falhou!!!")

[Teste do encapsulamento e desencapsulamento]
Chave = e055233254dfd82dd1584e43fbc949365b955ae1072e923331052691810a626b
Chave = e055233254dfd82dd1584e43fbc949365b955ae1072e923331052691810a626b
A chave desencapsulada é igual à resultante do encapsulamento!!!


In [42]:
# VER ISTO!!!
m = hashlib.sha3_256()
m.update(b"Nelson Faria")
representacaoBinaria = bin(int(m.hexdigest(), base=16))[2:].zfill(8)
print(representacaoBinaria)

representacaoHex = hex(int(representacaoBinaria, base=2))

print(m.hexdigest())
print(representacaoHex)
print('0x' + m.hexdigest() == representacaoHex)


1100111011101111000000011001001000100101101101000001110111000100011001111001110001100001111011110000001000101010101000010011110010001010100101100101000111110001010000101110011101101000101001110101000110100100001001111011010010111001101110010010011011111000
ceef019225b41dc4679c61ef022aa13c8a9651f142e768a751a427b4b9b926f8
0xceef019225b41dc4679c61ef022aa13c8a9651f142e768a751a427b4b9b926f8
True
